In [1]:
warehouses = {'A', 'B', 'C'}
numShops = 4
#Fixed transp cost
f = {('A', 1): 2000, ('A', 2):2500 ,  ('A', 3):1200, ('A', 4):5000 ,
                           ('B', 1):1000 , ('B', 2):2600 ,  ('B', 3):1500, ('B', 4):3000 ,
                            ('C', 1):900 , ('C', 2):2700 ,  ('C', 3):1100, ('C', 4): 1000 }

# Unit variable cost
c = {('A', 1): 3, ('A', 2):5 ,  ('A', 3):8, ('A', 4):10 ,
                           ('B', 1):11 , ('B', 2):12 ,  ('B', 3):6, ('B', 4):7 ,
                            ('C', 1):15 , ('C', 2):13 ,  ('C', 3):12, ('C', 4): 9 }

#Annual capacity for each warehouse
q = {'A':150, 'B':250, 'C':300}
# Annual demand for each shop
d = {1:110, 2:135, 3:195, 4:240}

In [2]:
import pyomo.environ as pe
import pyomo.opt as po

In [3]:
model = pe.ConcreteModel()
model.W = pe.Set(initialize = warehouses, ordered = False)
model.S = pe.RangeSet(1, numShops)

In [4]:
model.x = pe.Var(model.W, model.S, domain = pe.NonNegativeIntegers)
model.y = pe.Var(model.W, model.S, domain = pe.Binary)

In [5]:
objExpr = sum (model.x[w,s] * c[w,s] + model.y[w,s] * f[w,s]  for w in model.W for s in model.S)
model.obj = pe.Objective(expr = objExpr)

In [6]:
#Capacity constraints
model.conCapacity = pe.ConstraintList()
for w in model.W: 
    expr = sum(model.x[w,s]  for s in model.S)  <= q[w]
    model.conCapacity.add(expr)

#Demand constraints    
model.conDemand = pe.ConstraintList()
for s in model.S: 
    expr = sum(model.x[w,s]  for w in model.W)  == d[s]
    model.conDemand.add(expr)
#BigM constraints
model.conBigM = pe.ConstraintList()
for w in model.W:
    for s in model.S:
        expression = model.x[w,s] <= min(d[s], q[w]) * model.y[w,s]
        model.conBigM.add(expression)

In [7]:
solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee = True)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-12-10
Read LP format model from file C:\Users\rpo330\AppData\Local\Temp\tmp1h99gr3k.pyomo.lp
Reading time = 0.01 seconds
x25: 20 rows, 25 columns, 49 nonzeros
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 20 rows, 25 columns and 49 nonzeros
Model fingerprint: 0x1ed0914c
Variable types: 1 continuous, 24 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [3e+00, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective 18685.000000
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 19 rows, 24 columns, 48 nonzeros
Variable types: 0 continuous, 24 integer (12 binary)
Found heuristic solution: objective 18677.000000

Root relaxation: objective 1.135500e+04, 19 iterations, 0.00 second

In [8]:
print(result.solver.status)
print(result.solver.termination_condition)

ok
optimal


In [9]:
print("Total cost = "+ str(pe.value(model.obj)))
print("Optimal solution:")
for w in model.W: 
    for s in model.S:
        if(pe.value(model.y[w,s]) > 0):
            print(str(model.y[w,s])+" = "+str(pe.value(model.y[w,s])))
        if(pe.value(model.x[w,s]) > 0):
             print(str(model.x[w,s])+" = "+str(pe.value(model.x[w,s])))

Total cost = 12335.0
Optimal solution:
y[C,1] = 1.0
x[C,1] = 55.0
y[C,4] = 1.0
x[C,4] = 240.0
y[A,2] = 1.0
x[A,2] = 135.0
y[B,1] = 1.0
x[B,1] = 55.0
y[B,3] = 1.0
x[B,3] = 195.0
